In [11]:
# 셀1

import os

# CSV 파일 경로 (가장 중요한 부분! Windows 경로는 문자열 앞에 r을 붙여주세요)
csv_file_path = r"C:\Users\redsk\OneDrive\デスクトップ\mikael_project\klook\data\아시아\일본\일본_통합_klook_products.csv"

# 모드 설정: "range" 또는 "all"
mode = "range"  # 범위 지정 모드
# mode = "all"    # 전체 처리 모드

# Range 모드 설정 (번호 컬럼값 기준)
start_num = 4   # 시작 번호 컬럼값
end_num = 7     # 끝 번호 컬럼값

# --- 경로 존재 여부 확인 ---#
# 경로를 처음부터 올바르게 입력했으므로 별도의 변환 함수는 필요 없습니다.
if os.path.exists(csv_file_path):
    print(f"✅ 파일 경로 확인됨: {csv_file_path}")
else:
    print(f"❌ 파일을 찾을 수 없습니다: {csv_file_path}")
    print("💡 경로를 다시 확인해주세요.")

# --- 초기 패키지 설치(아래 코드가 안될때도 실행) ---
# (240424) 코드 일부 업데이트되었습니다.

# !python -m pip install --upgrade pip
# !pip uninstall selenium -y
# !pip install selenium==4.25.0

# !pip uninstall webdriver-manager -y
# !pip install webdriver-manager -U

# !pip uninstall undetected_chromedriver -y
# !pip install undetected-chromedriver -U
# !pip install user_agents

# !pip install pyperclip

# !pip install chromedriver_autoinstaller

# --- 라이브러리 import ---
import pandas as pd
# DataFrame을 컨트롤하는 라이브러리

import warnings, os, time, shutil, urllib, random
# warnings : 파이썬 경고 제어
# os : 경로 또는 파일 제어
# time : 시간 제어(일정시간 delay등)
# shutil : os의 폴더 제어(생성/삭제)
# urllib : 특정 url에서 이미지를 받을때 사용

warnings.filterwarnings(action='ignore')
# 파이썬에서 불필요한 warning은 안뜨게 함

from PIL import Image
# 파이썬에서 이미지를 다룰때 사용(png등을 불러들여서 리사이즈 저장 등)

from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
# Keys.ARROW_DOWN, Keys.ARROW_LEFT, Keys.ARROW_RIGHT, Keys.ARROW_UP, Keys.BACK_SPACE,
# Keys.CONTROL, Keys.ALT, Keys.DELETE,
# Keys.ENTER, Keys.SHIFT, Keys.SPACE, Keys.TAB, Keys.EQUALS, Keys.ESCAPE, Keys.HOME,
# Keys.INSERT, Keys.PAGE_UP,
# Keys.PAGE_DOWN, Keys.F1, Keys.F2, Keys.F3, Keys.F4, Keys.F5, Keys.F6, Keys.F7, Keys.F8,
# Keys.F9, Keys.F10, Keys.F11, Keys.F12
from selenium.webdriver import ActionChains

# --- 크롬 드라이버 실행 ---
import chromedriver_autoinstaller, random, os
chromedriver_autoinstaller.install()

import undetected_chromedriver as uc
from user_agents import parse

import selenium
print(selenium.__version__)

import pyperclip

def make_user_agent(ua, is_mobile):
    user_agent = parse(ua)
    model = user_agent.device.model
    platform = user_agent.os.family
    platform_version = user_agent.os.version_string + ".0.0"
    version = user_agent.browser.version[0]
    ua_full_version = user_agent.browser.version_string
    architecture = "x86"
    print(platform)
    if is_mobile:
        platform_info = "Linux armv8l"
        architecture = ""
    else:  # Windows
        platform_info = "Win32"
        model = ""
    RET_USER_AGENT = {
        "appVersion": ua.replace("Mozilla/", ""),
        "userAgent": ua,
        "platform": f"{platform_info}",
        "acceptLanguage": "ko-KR, kr, en-US, en",
        "userAgentMetadata": {
            "brands": [
                {"brand": "Google Chrome", "version": f"{version}"},
                {"brand": "Chromium", "version": f"{version}"},
                {"brand": " Not A;Brand", "version": "99"}
            ],
            "fullVersionList": [
                {"brand": "Google Chrome", "version": f"{version}"},
                {"brand": "Chromium", "version": f"{version}"},
                {"brand": " Not A;Brand", "version": "99"}
            ],
            "fullVersion": f"{ua_full_version}",
            "platform": platform,
            "platformVersion": platform_version,
            "architecture": architecture,
            "model": model,
            "mobile": is_mobile  # True, False
        }
    }
    return RET_USER_AGENT

def generate_random_geolocation():
    ltop_lat = 37.75415601640249
    ltop_long = 126.86767642302573
    rbottom_lat = 37.593829172663945
    rbottom_long = 127.15276051439332

    targetLat = random.uniform(rbottom_lat, ltop_lat)
    targetLong = random.uniform(ltop_long, rbottom_long)
    return {"latitude": targetLat, "longitude": targetLong, "accuracy": 100}

# --- 유틸리티 함수들 ---
def get_url_by_number(number):
    """
    특정 번호에 해당하는 URL을 CSV 파일에서 찾아 반환합니다. (수정된 버전)
    - 여러 인코딩으로 파일 읽기를 먼저 시도합니다.
    - 파일 읽기 성공 후, 데이터의 유효성을 검사합니다.
    """
    encodings = ['utf-8', 'cp949', 'euc-kr', 'utf-8-sig']

    for encoding in encodings:
        try:
            # 1. 특정 인코딩으로 파일 읽기 시도
            df = pd.read_csv(csv_file_path, encoding=encoding)
            
            # --- 여기부터 중요 ---
            # 파일 읽기에 성공했다면, 이 파일의 내용이 최종 데이터입니다.
            # 이 데이터를 기준으로 URL을 찾고, 결과가 있든 없든 함수를 종료해야 합니다.
            
            row = df[df['번호'] == number]
            
            # URL 값이 유효하게 존재하는지 검사
            if not row.empty and 'URL' in row.columns and pd.notna(row['URL'].iloc[0]) and str(row['URL'].iloc[0]).strip():
                return str(row['URL'].iloc[0])
            else:
                # 파일은 올바르게 읽었지만, 해당 번호의 URL이 없거나 비어있는 경우.
                # 이것이 최종 결과이므로 None을 반환하고 함수를 종료합니다.
                return None 
                
        except UnicodeDecodeError:
            # 현재 인코딩이 맞지 않으면 이 에러가 발생합니다.
            # 다음 인코딩으로 시도를 계속합니다.
            continue
        except Exception as e:
            # 파일을 찾지 못했거나 다른 문제가 발생한 경우
            print(f"URL 가져오기 오류 (번호 {number}) 인코딩 '{encoding}': {e}")
            return None

    # for 루프가 모두 실행되었다면, 모든 인코딩으로 파일 읽기에 실패한 것입니다.
    print(f"❌ 모든 인코딩 시도 실패. URL을 가져올 수 없습니다 (번호 {number}). 파일 경로와 인코딩을 확인하세요.")
    return None

def save_ad_link(number, ad_link):
    """
    생성된 광고 링크를 CSV에 저장합니다. (파일/행 자동 생성 기능 추가 버전)
    - 파일이 없으면 새로 생성하고, 해당 번호의 행이 없으면 새로 추가합니다.
    """
    encodings = ['utf-8', 'cp949', 'euc-kr', 'utf-8-sig']
    df = None

    # 1. 여러 인코딩으로 파일 읽기를 시도
    for encoding in encodings:
        try:
            df = pd.read_csv(csv_file_path, encoding=encoding)
            # 파일 읽기에 성공하면 루프를 중단합니다.
            break
        except UnicodeDecodeError:
            # 인코딩이 맞지 않으면 다음 인코딩으로 넘어갑니다.
            continue
        except FileNotFoundError:
            # 파일이 없으면 '번호', 'URL', 'klook_ad_link' 컬럼을 가진 새 DataFrame을 생성합니다.
            print("알림: CSV 파일을 찾을 수 없어 새로 생성합니다.")
            df = pd.DataFrame(columns=['번호', 'URL', 'klook_ad_link'])
            break

    # 파일을 성공적으로 읽거나 새로 만들지 못했다면 작업을 중단합니다.
    if df is None:
        print(f"❌ 모든 인코딩으로 파일 읽기 실패. 저장할 수 없습니다 (번호 {number})")
        return False

    # 2. DataFrame 수정 및 저장
    try:
        # 'klook_ad_link' 컬럼이 없으면 새로 추가합니다.
        if 'klook_ad_link' not in df.columns:
            df['klook_ad_link'] = ''

        # '번호'가 일치하는 행의 인덱스를 찾습니다.
        row_index_list = df.index[df['번호'] == number].tolist()

        if row_index_list:
            # 기존 행이 있으면 해당 행의 'klook_ad_link' 값을 업데이트합니다.
            df.loc[row_index_list[0], 'klook_ad_link'] = ad_link
        else:
            # 기존 행이 없으면 새로운 행으로 추가합니다.
            new_row = pd.DataFrame([{'번호': number, 'klook_ad_link': ad_link}])
            df = pd.concat([df, new_row], ignore_index=True)

        # 엑셀과의 호환성을 위해 항상 'utf-8-sig'로 저장합니다.
        df.to_csv(csv_file_path, index=False, encoding='utf-8-sig')
        print(f"✅ 번호 {number} 저장 완료: {ad_link[:50]}...")
        return True
    except Exception as e:
        print(f"❌ 데이터 저장 중 오류 발생 (번호 {number}): {e}")
        return False

def is_already_processed(number):
    """
    특정 번호가 이미 처리되었는지 CSV 파일을 확인하여 True/False를 반환합니다. (수정된 버전)
    - 여러 인코딩으로 파일 읽기를 시도하여 안정성을 높였습니다.
    - 'klook_ad_link' 컬럼의 값이 유효한지(비어있지 않은지) 검사합니다.
    """
    encodings = ['utf-8', 'cp949', 'euc-kr', 'utf-8-sig']
    
    for encoding in encodings:
        try:
            df = pd.read_csv(csv_file_path, encoding=encoding)

            # 파일 읽기에 성공했으므로, 이 데이터를 기준으로 최종 결과를 판단하고 함수를 종료합니다.
            if 'klook_ad_link' in df.columns:
                row = df[df['번호'] == number]
                if not row.empty:
                    ad_link = row['klook_ad_link'].iloc[0]
                    # 링크 값이 유효한지(NaN이 아니고, 공백만 있는 문자열이 아닌지) 확인합니다.
                    return pd.notna(ad_link) and str(ad_link).strip() != ''

            # 'klook_ad_link' 컬럼이 없거나, 해당 번호의 행이 없으면 '미처리' 상태입니다.
            return False

        except UnicodeDecodeError:
            # 인코딩이 맞지 않으면 다음 인코딩으로 넘어갑니다.
            continue
        except FileNotFoundError:
            # 파일이 존재하지 않으면 '미처리' 상태로 간주합니다.
            return False
        except Exception as e:
            # [수정] f-string이 여러 줄에 걸쳐 있던 문법 오류를 수정했습니다.
            print(f"처리 상태 확인 오류 (번호 {number}) 인코딩 '{encoding}': {e}")
            # 오류 발생 시에도 안전하게 '미처리'로 판단합니다.
            return False

    # 모든 인코딩으로 파일 읽기에 실패한 경우
    print(f"❌ 모든 인코딩 시도 실패. 처리 상태 확인 불가 (번호 {number})")
    return False

def get_processing_list():
    """
    처리할 번호 리스트를 생성합니다. (성능 개선 버전)
    - 'all' 모드에서 iterrows() 대신 벡터 연산을 사용하여 처리 속도를 높였습니다.
    """
    if mode == "range":
        # range 모드는 간단하고 효율적이므로 그대로 사용합니다.
        return list(range(start_num, end_num + 1))
        
    elif mode == "all":
        encodings = ['utf-8', 'cp949', 'euc-kr', 'utf-8-sig']
        
        for encoding in encodings:
            try:
                df = pd.read_csv(csv_file_path, encoding=encoding)

                # 파일 읽기 성공. 이제 데이터를 확인하고 결과를 확정합니다.
                
                # '번호'와 'URL' 컬럼이 모두 존재하는지 먼저 확인합니다.
                if '번호' not in df.columns or 'URL' not in df.columns:
                    print(f"경고: '{encoding}'으로 읽은 파일에 '번호' 또는 'URL' 컬럼이 없습니다.")
                    # 컬럼이 없으면 유효한 데이터가 없으므로 빈 리스트를 반환합니다.
                    return []

                # --- [성능 개선] 벡터 연산을 사용하여 유효한 행을 한 번에 필터링 ---
                mask = (
                    df['번호'].notna() &
                    df['URL'].notna() &
                    (df['URL'].astype(str).str.strip() != '')
                )
                
                # 마스크를 적용하여 유효한 번호만 추출하고, 정수 리스트로 변환합니다.
                valid_numbers = df.loc[mask, '번호'].astype(int).tolist()
                
                # 최종 목록을 정렬하여 반환합니다.
                return sorted(valid_numbers)

            except UnicodeDecodeError:
                # 인코딩이 맞지 않으면 다음으로 넘어갑니다.
                continue
            except FileNotFoundError:
                print("처리 리스트 생성 오류: CSV 파일을 찾을 수 없습니다.")
                return []
            except Exception as e:
                print(f"처리 리스트 생성 오류 인코딩 '{encoding}': {e}")
                return []
        
        # 모든 인코딩으로 파일 읽기에 실패한 경우
        print("❌ 모든 인코딩 시도 실패. 처리 리스트 생성 불가")
        return []
        
    # mode가 'range'도 'all'도 아닌 경우
    return []


def safe_find_element_and_action(driver, xpath, action_type="click", text=None, max_retries=3):
    """안전한 요소 찾기 및 액션 수행"""
    for attempt in range(max_retries):
        try:
            elem = driver.find_element("xpath", xpath)
            if action_type == "click":
                elem.click()
            elif action_type == "clear":
                elem.clear()
            elif action_type == "send_keys" and text:
                elem.clear()
                elem.send_keys(text)
            time.sleep(random.uniform(2, 4))
            return True
        except Exception as e:
            print(f"액션 시도 {attempt + 1}/{max_retries} 실패: {e}")
            if attempt < max_retries - 1:
                time.sleep(random.uniform(1, 2))
            else:
                return False
    return False

def find_next_number(current_num, processing_list, is_processed_func):
    """현재 번호 다음의 미처리 번호를 찾아서 반환합니다."""
    try:
        current_index = processing_list.index(current_num)
        for i in range(current_index + 1, len(processing_list)):
            candidate = processing_list[i]
            if not is_processed_func(candidate):
                return candidate
        return None
    except ValueError:
        return None


options = uc.ChromeOptions()

UA = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36 Edg/122.0.0.0"
options.add_argument(f"--user-agent={UA}")
# User Agent 확인 : https://www.whatismybrowser.com/detect/what-is-my-user-agent/
# [중요] 위에 링크에서 나오는 값을 UA에 넣어줘야 함

rand_user_folder = random.randrange(1, 100)
raw_path = os.path.abspath("cookies")
try:
    shutil.rmtree(raw_path)
except:
    pass
os.makedirs(raw_path, exist_ok=True)
user_cookie_name = f"{raw_path}/{rand_user_folder}"
if os.path.exists(user_cookie_name) == False:
    os.makedirs(user_cookie_name, exist_ok=True)
# options.user_data_dir = user_cookie_name
# 쿠키 랜덤 설정

try:
    driver = uc.Chrome(user_data_dir=user_cookie_name, options=options)
except Exception as e:
    print('\n', "-"*50, "\n", "-"*50, "\n")
    print("# 키홈 메세지 : 혹시 여기서 에러 발생시 [아래 블로그 참고 -> 재부팅 -> 다시 코드실행] 해보시길 바랍니다! \n (구글크롬 버젼 업그레이드 문제)")
    print('https://appfollow.tistory.com/102')  # 에러 메시지를 출력
    print('\n', "-"*50, "\n", "-"*50, "\n")
    raise RuntimeError

UA_Data = make_user_agent(UA, False)
driver.execute_cdp_cmd("Network.setUserAgentOverride", UA_Data)
# user agent data 설정

GEO_DATA = generate_random_geolocation()
driver.execute_cdp_cmd("Emulation.setGeolocationOverride", GEO_DATA)
driver.execute_cdp_cmd("Emulation.setUserAgentOverride", UA_Data)
driver.execute_cdp_cmd("Emulation.setNavigatorOverrides", {"platform": "Linux armv8l"})

# 전역 변수
current_processing_number = None
consecutive_errors = 0

print(f"설정 완료!")
print(f"CSV 파일: {csv_file_path}")
print(f"모드: {mode}")
if mode == "range":
    print(f"처리 범위: {start_num}번 ~ {end_num}번")
print(f"처리 대상: {get_processing_list()}")

✅ 파일 경로 확인됨: C:\Users\redsk\OneDrive\デスクトップ\mikael_project\klook\data\아시아\일본\일본_통합_klook_products.csv
4.9.1
Windows
설정 완료!
CSV 파일: C:\Users\redsk\OneDrive\デスクトップ\mikael_project\klook\data\아시아\일본\일본_통합_klook_products.csv
모드: range
처리 범위: 4번 ~ 7번
처리 대상: [4, 5, 6, 7]


In [12]:
# 셀 2
driver.get("https://affiliate.klook.com/ko/")
time.sleep(random.uniform(3,5))
#클룩제휴파트너 사이트 접속

In [13]:
# 셀 3
elem = driver.find_element("xpath", "/html/body/div/div[1]/div[1]/div[3]/div[4]/span")
elem.click()
time.sleep(random.uniform(3,5))
# 로그인 클릭

elem = driver.find_element("xpath", "/html/body/div[1]/div[1]/div[1]/div[4]/div/div[2]/div/div[2]/div[2]/form/div[1]/div[2]/div/input")
elem.send_keys("mikael227@gmail.com")
time.sleep(random.uniform(3,5))
# 이메일 박스클릭 후 입력

elem = driver.find_element("xpath", "/html/body/div[1]/div[1]/div[1]/div[4]/div/div[2]/div/div[2]/div[2]/form/div[2]/div[2]/div/input")
elem.send_keys("osaka2002.")
time.sleep(random.uniform(3,5))
# 비밀번호 박스클릭 후 입력

elem = driver.find_element("xpath", "/html/body/div[1]/div[1]/div[1]/div[4]/div/div[2]/div/div[2]/div[2]/form/div[3]/div[2]/label/span[1]/input")
elem.click()
time.sleep(random.uniform(3,5))
# 체크박스 클릭

elem = driver.find_element("xpath", "/html/body/div[1]/div[1]/div[1]/div[4]/div/div[2]/div/div[2]/div[2]/form/div[4]/div[2]/div[2]/div[1]/label/span[1]/input")
elem.click()
time.sleep(random.uniform(3,5))
# 로그인 상태유지 클릭

elem = driver.find_element("xpath", "/html/body/div[1]/div[1]/div[1]/div[4]/div/div[2]/div/div[2]/div[3]/button")
elem.click()
time.sleep(random.uniform(3,5))
# 로그인버튼 클릭

In [14]:
# 셀 4
elem = driver.find_element("xpath", "/html/body/div/div[2]/div[1]/div[1]/div[2]/div/div[2]/div[1]/div/div/div[1]")
elem.click()
time.sleep(random.uniform(3,5))
#텍스트 링크 클릭

In [15]:
# 셀 5
elem = driver.find_element("xpath", "/html/body/div/div[2]/div[1]/div[3]/div[3]/form/div[2]/div[2]/div/input")
elem.clear()
time.sleep(random.uniform(3,5))
# 상품 url박스 클릭 후 클리어

In [16]:
# 셀 6
# 현재 처리할 번호의 URL 가져오기

if current_processing_number is None:
    processing_list = get_processing_list()
    if processing_list:
        current_processing_number = processing_list[0]
        print(f"🚀 처리 시작: 번호 {current_processing_number}")

# 현재 번호의 URL 가져와서 입력
if current_processing_number is not None:
    current_url = get_url_by_number(current_processing_number)
    if current_url:
        print(f"🔗 번호 {current_processing_number} URL 입력 중: {current_url[:60]}...")
        success = safe_find_element_and_action(
            driver,
            "/html/body/div/div[2]/div[1]/div[3]/div[3]/form/div[2]/div[2]/div/input",
            "send_keys",
            current_url
        )
        if success:
            print(f"✅ 번호 {current_processing_number} URL 입력 완료")
        else:
            print(f"❌ URL 입력 실패")
    else:
        print(f"❌ 번호 {current_processing_number}의 URL을 찾을 수 없음")
else:
    print("❌ 처리할 번호가 없습니다.")

🚀 처리 시작: 번호 4
🔗 번호 4 URL 입력 중: https://www.klook.com/ko/activity/148089-hokkaido-shrine-jyo...
✅ 번호 4 URL 입력 완료


In [17]:
# 셀 7
elem = driver.find_element("xpath", "/html/body/div/div[2]/div[1]/div[3]/div[3]/div/button[1]")
elem.click()
time.sleep(random.uniform(3,5))
# 저장 및 링크생성 클릭

In [18]:
#셀 8
if current_processing_number is not None:
    print(f"📋 번호 {current_processing_number} URL 복사 중...")

    COPY_BUTTON_SELECTOR = "/html/body/div/div[2]/div[1]/div[3]/div[3]/div/div/div[1]/div[2]/div/div[1]/div/button"

    success = safe_find_element_and_action(driver, COPY_BUTTON_SELECTOR, "click")

    if success:
        time.sleep(random.uniform(1, 2))
        try:
            generated_url = pyperclip.paste()
            if generated_url and generated_url.strip():
                print(f"📋 복사된 URL: {generated_url[:50]}...")

                if save_ad_link(current_processing_number, generated_url):
                    print(f"💾 번호 {current_processing_number} 저장 완료!")

                    # 다음 처리 번호 찾기
                    processing_list = get_processing_list()
                    next_number = find_next_number(
                        current_processing_number, processing_list, is_already_processed
                    )

                    # 전역변수 업데이트
                    if next_number is not None:
                        globals()['current_processing_number'] = next_number
                        print(f"➡️ 다음 처리 번호: {next_number}")
                    else:
                        print("🏁 모든 처리 완료!")
                        globals()['current_processing_number'] = None
                else:
                    print(f"❌ 번호 {current_processing_number} 저장 실패!")
            else:
                print("❌ 클립보드에서 URL을 가져오지 못했습니다.")
        except Exception as e:
            print(f"❌ 클립보드 오류: {e}")
    else:
        print("❌ URL 복사 버튼 클릭 실패")
else:
    print("❌ 처리 중인 번호가 없습니다.")

📋 번호 4 URL 복사 중...
📋 복사된 URL: https://affiliate.klook.com/redirect?aid=89627&aff...
✅ 번호 4 저장 완료: https://affiliate.klook.com/redirect?aid=89627&aff...
💾 번호 4 저장 완료!
➡️ 다음 처리 번호: 5


In [19]:
 #셀 9
elem = driver.find_element("xpath", "/html/body/div/div[2]/div[1]/div[3]/div[3]/div/button[2]")
elem.click()
time.sleep(random.uniform(3,5))
#새 링크 생성 클릭

In [20]:
#셀 10

# --- 선택자(XPath)를 상수로 정의하여 관리 용이성 증대 ---
URL_INPUT_BOX = "/html/body/div/div[2]/div[1]/div[3]/div[3]/form/div[2]/div[2]/div/input"
CREATE_LINK_BTN = "/html/body/div/div[2]/div[1]/div[3]/div[3]/div/button[1]"
COPY_URL_BTN = "/html/body/div/div[2]/div[1]/div[3]/div[3]/div/div/div[1]/div[2]/div/div[1]/div/button"
CREATE_NEW_LINK_BTN = "/html/body/div/div[2]/div[1]/div[3]/div[3]/div/button[2]"

# --- 완전 자동화 반복 루프 ---
print("🚀 자동화 시작!")

while current_processing_number is not None:
    try:
        print(f"\n=== 번호 {current_processing_number} 처리 중 ===")

        # 1. URL 박스 클리어
        print("🧹 URL 박스 클리어 중...")
        success = safe_find_element_and_action(driver, URL_INPUT_BOX, "clear")
        if not success:
            print(f"❌ 번호 {current_processing_number}: URL 박스 클리어 실패. 다음 번호로 넘어갑니다.")
            # 실패 시 다음 번호를 찾아 바로 루프의 다음 순회로 넘어감
            processing_list = get_processing_list()
            current_processing_number = find_next_number(current_processing_number, processing_list, is_already_processed)
            continue

        # 2. URL 입력
        current_url = get_url_by_number(current_processing_number)
        if not current_url:
            print(f"❌ 번호 {current_processing_number}: URL을 찾을 수 없음. 다음 번호로 넘어갑니다.")
            processing_list = get_processing_list()
            current_processing_number = find_next_number(current_processing_number, processing_list, is_already_processed)
            continue
            
        print(f"🔗 URL 입력 중: {current_url[:50]}...")
        safe_find_element_and_action(driver, URL_INPUT_BOX, "send_keys", current_url)

        # 3. 링크 생성
        print("⚡ 링크 생성 중...")
        safe_find_element_and_action(driver, CREATE_LINK_BTN, "click")
        
        # (중요) 링크 생성 후 결과가 로딩될 때까지 잠시 대기하는 것이 좋습니다.
        # time.sleep() 대신 WebDriverWait를 사용하는 것이 더 안정적입니다.
        time.sleep(random.uniform(2, 3)) 

        # 4. URL 복사
        print("📋 URL 복사 및 저장 중...")
        success = safe_find_element_and_action(driver, COPY_URL_BTN, "click")
        if not success:
            print(f"❌ 번호 {current_processing_number}: URL 복사 버튼 클릭 실패.")
            # 복사 실패 시에도 '새 링크 생성'은 시도해야 할 수 있으므로 continue 대신 다음 로직 진행
        else:
            time.sleep(1)
            generated_url = pyperclip.paste()
            if generated_url and save_ad_link(current_processing_number, generated_url):
                print(f"✅ 번호 {current_processing_number} 완료!")
            else:
                print(f"❌ 번호 {current_processing_number}: URL을 클립보드에서 가져오거나 저장하는 데 실패했습니다.")

        # 다음 번호로 이동 준비
        processing_list = get_processing_list()
        next_number = find_next_number(current_processing_number, processing_list, is_already_processed)
        current_processing_number = next_number # globals() 대신 직접 할당

        # 5. 새 링크 생성 (다음 루프를 위해)
        if current_processing_number is not None:
            print("🔄 새 링크 생성 버튼 클릭 (다음 작업을 위해)...")
            safe_find_element_and_action(driver, CREATE_NEW_LINK_BTN, "click")
            time.sleep(random.uniform(1, 2)) # 페이지가 초기화될 시간을 줌

    except Exception as e:
        print(f"💥 예상치 못한 오류 발생: {e}")
        print("스크립트를 중단합니다.")
        break

if current_processing_number is None:
    print("\n🏁 모든 처리 완료!")

🚀 자동화 시작!

=== 번호 5 처리 중 ===
🧹 URL 박스 클리어 중...
🔗 URL 입력 중: https://www.klook.com/ko/activity/156805-premium-b...
⚡ 링크 생성 중...
📋 URL 복사 및 저장 중...
✅ 번호 5 저장 완료: https://affiliate.klook.com/redirect?aid=89627&aff...
✅ 번호 5 완료!
🔄 새 링크 생성 버튼 클릭 (다음 작업을 위해)...

=== 번호 6 처리 중 ===
🧹 URL 박스 클리어 중...
🔗 URL 입력 중: https://www.klook.com/ko/activity/3067-7-day-jr-ho...
⚡ 링크 생성 중...
📋 URL 복사 및 저장 중...
✅ 번호 6 저장 완료: https://affiliate.klook.com/redirect?aid=89627&aff...
✅ 번호 6 완료!
🔄 새 링크 생성 버튼 클릭 (다음 작업을 위해)...

=== 번호 7 처리 중 ===
🧹 URL 박스 클리어 중...
🔗 URL 입력 중: https://www.klook.com/ko/activity/112369-hot-sprin...
⚡ 링크 생성 중...
📋 URL 복사 및 저장 중...
✅ 번호 7 저장 완료: https://affiliate.klook.com/redirect?aid=89627&aff...
✅ 번호 7 완료!

🏁 모든 처리 완료!
